## Exercise 57 - Shortest Path

In [1]:
from graphframes import GraphFrame

inputPathVertexes = "/data/students/bigdata-01QYD/ex_data/Ex57/data/vertexes.csv"
inputPathEdges = "/data/students/bigdata-01QYD/ex_data/Ex57/data/edges.csv"
outputPath = "resOut_ex57/"

In [2]:
verticesDF = spark.read.load(inputPathVertexes, format="csv", header=True, inferSchema=True)
edgesDF = spark.read.load(inputPathEdges, format="csv", header=True, inferSchema=True)

In [3]:
verticesDF.show()

+---+-----+---+
| id| name|age|
+---+-----+---+
| u1|Alice| 34|
| u2|  Bob| 36|
| u3| John| 30|
| u4|David| 29|
| u5| Paul| 32|
| u6| Adel| 36|
| u7| Eddy| 60|
+---+-----+---+



In [4]:
edgesDF.show()

+---+---+--------+
|src|dst|linktype|
+---+---+--------+
| u1| u2|  friend|
| u1| u4|  friend|
| u1| u5|  friend|
| u2| u1|  friend|
| u2| u3|  follow|
| u3| u2|  follow|
| u4| u1|  friend|
| u4| u5|  friend|
| u5| u1|  friend|
| u5| u4|  friend|
| u5| u6|  follow|
| u6| u3|  follow|
+---+---+--------+



Output:
* Select the users who can reach user u1 in less than 3 hops (i.e., at most two edges)
    * Do not consider u1 itself
* For each of the selected users, store in the output folder his/her name and the minimum number of hops to reach user u1
    * One user per line
    * Format: user name, #hops to user u1
* Use the CSV format to store the result

In [7]:
g = GraphFrame(verticesDF, edgesDF)

shortestPathsLength = g.shortestPaths(['u1'])
shortestPathsLength.show()

+---+-----+---+---------+
| id| name|age|distances|
+---+-----+---+---------+
| u6| Adel| 36|[u1 -> 3]|
| u3| John| 30|[u1 -> 2]|
| u2|  Bob| 36|[u1 -> 1]|
| u4|David| 29|[u1 -> 1]|
| u5| Paul| 32|[u1 -> 1]|
| u1|Alice| 34|[u1 -> 0]|
| u7| Eddy| 60|       []|
+---+-----+---+---------+



In [11]:
filteredShortestPathsLength = shortestPathsLength.filter("distances.u1 < 3 AND id <> 'u1'")
filteredShortestPathsLength.show()

+---+-----+---+---------+
| id| name|age|distances|
+---+-----+---+---------+
| u3| John| 30|[u1 -> 2]|
| u2|  Bob| 36|[u1 -> 1]|
| u4|David| 29|[u1 -> 1]|
| u5| Paul| 32|[u1 -> 1]|
+---+-----+---+---------+



In [13]:
cleanedOutput = filteredShortestPathsLength.selectExpr("name","distances['u1'] as numHops")
cleanedOutput.show()

+-----+-------+
| name|numHops|
+-----+-------+
| John|      2|
|  Bob|      1|
|David|      1|
| Paul|      1|
+-----+-------+



In [14]:
cleanedOutput.write.csv(outputPath, header=True)